In [1]:
import os
import numpy as np
from PIL import Image
import csv
import pandas as pd

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
# from facenet_pytorch import MTCNN, InceptionResnetV1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp drive/'My Drive'/facerecog/age_prediction.zip .
!unzip -q age_prediction.zip

replace age_prediction/test/001/7148.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
# !kaggle competitions download -c idl-fall21-hw2p2s1-face-classification

In [5]:
# !unzip idl-fall21-hw2p2s1-face-classification

In [6]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label

In [7]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n

In [8]:
img_list, label_list, class_n = parse_data('./age_prediction/train')

#Images		#Labels
185632		100


In [9]:
trainset = ImageDataset(img_list, label_list)

In [10]:
train_data_item, train_data_label = trainset.__getitem__(0)

In [11]:
print('data item shape: {}\t data item label: {}'.format(train_data_item.shape, train_data_label))

data item shape: torch.Size([3, 128, 128])	 data item label: 43


In [12]:
dataloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=1, drop_last=False)

## Torchvision DataSet and DataLoader

In [13]:
imageFolder_dataset = torchvision.datasets.ImageFolder(root='age_prediction/train/', 
                                                       transform=torchvision.transforms.ToTensor())

In [14]:
imageFolder_dataloader = DataLoader(imageFolder_dataset, batch_size=10, shuffle=True, num_workers=1)

In [15]:
print(imageFolder_dataset.__len__(), len(imageFolder_dataset.classes))

185632 100


## ResNet Class

In [23]:
"""

This cell is credited to the SOURCE CODE FOR TORCHVISION.MODELS.RESNET

URL: https://pytorch.org/vision/0.8/_modules/torchvision/models/resnet.html

"""

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [24]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=100, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model

def resnet34(progress=True, **kwargs):
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], **kwargs)

# Datasets and DataLoaders

In [25]:
"""
model = resnet34(pretrained=False, progress=True, num_classes = 4000)
"""
#Above is the model from classes above
from torchvision import transforms

myTrainTransforms = transforms.Compose([transforms.Resize((224,224)),
                                  # transforms.RandomRotation(20),
                                  transforms.ColorJitter(hue=.05,saturation=.05, brightness = 0.3),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomAffine(20),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

myTestTransforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = torchvision.datasets.ImageFolder(root='age_prediction/train', 
                                                 transform=myTrainTransforms) #.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='age_prediction/test/', 
                                               transform= myTestTransforms)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=128, 
                                             shuffle=False, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [26]:
# Image.open(train_dataset[0])

# Instantiation and Initializations

In [27]:
numEpochs = 100
in_features = 3 # RGB channels

learningRate = 0.1
weightDecay = 5e-5 

num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet34(progress=True, num_classes = 100)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), nesterov =True, lr=learningRate, weight_decay=weightDecay, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7, patience=1, verbose =True)

# Training the Model

In [28]:
# model.load_state_dict(torch.load("model_29.pt"))

In [29]:
# Train!
epoch = 0
best_loss = np.inf
for epoch in range(epoch, numEpochs):
    
    # Train
    model.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if (batch_num+1) % 1000 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/5000))
            avg_loss = 0.0
        
        if((epoch+1)%20 == 0):
            for group in optimizer.param_groups:
                group['lr'] /=2
        
        # Saving the model
        # saved_model_name = "model_"+str(epoch)+".ckpt"
        # torch.save(model.state_dict(), saved_model_name)
    
    # Validate
    model.eval()
    num_correct = 0
    avg_val_loss = 0
    for batch_num, (x, y) in enumerate(dev_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        val_loss = criterion(outputs, y.long())
        avg_val_loss += val_loss.item()
    
    accuracy = num_correct / len(dev_dataset)
    avg_val_loss /= len(dev_dataset)
    lr_scheduler.step(avg_val_loss)
    
    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch+1, num_correct / len(dev_dataset)))
    if avg_val_loss < best_loss:
        torch.save(model.state_dict(), 'best_model.pt')
        checkpoint = { 
                        'epoch': epoch,
                        'model': model,
                        'optimizer': optimizer,
                        'lr_sched': lr_scheduler}
        torch.save(checkpoint, 'best_checkpoint_models.pt')
        best_loss = avg_val_loss

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 1	Batch: 1000	Avg-Loss: 0.8097


RuntimeError: ignored

# Resuming training from saved model

####                 In case it is needed to resume training instead of starting training from 0th epoch

In [ ]:
# # model = Net()
# # optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

        # checkpoint = torch.load("checkpoint_model_27.pt")
        # model = checkpoint['model']
        # optimizer = checkpoint['optimizer']
        # epoch = checkpoint['epoch']
# # loss = checkpoint['loss']

In [ ]:
epoch = 0
while(epoch < numEpochs):
    
    # Train
    model.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()

        x, y = x.to(device), y.to(device)

        outputs = model(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if (batch_num+1) % 1000 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/5000))
            avg_loss = 0.0

        if((epoch+1)%20 == 0):
            for group in optimizer.param_groups:
                group['lr'] /=2

        # Saving the model
        # saved_model_name = "model_"+str(epoch)+".ckpt"
        # torch.save(model.state_dict(), saved_model_name)
    
    # Validate
    model.eval()
    num_correct = 0
    avg_val_loss = 0
    for batch_num, (x, y) in enumerate(dev_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        val_loss = criterion(outputs, y.long())
        avg_val_loss += val_loss.item()

    accuracy = num_correct / len(dev_dataset)
    avg_val_loss /= len(dev_dataset)
    lr_scheduler.step(avg_val_loss)

    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch+1, num_correct / len(dev_dataset)))
    # if epoch%5==0:
    torch.save(model.state_dict(), 'model_' + str(epoch+1) + '.pt')
    checkpoint = { 
                    'epoch': epoch,
                    'model': model,
                    'optimizer': optimizer,
                    'lr_sched': lr_scheduler}
    torch.save(checkpoint, 'checkpoint_modelss_' + str(epoch+1) + '.pt')
    epoch +=1

In [ ]:
optimizer.state_dict

In [ ]:
train_dataset.class_to_idx

In [ ]:
checkpoint2 = torch.load('checkpoint_modelss_1.pt')
model = resnet34(pretrained=False, progress=True, num_classes = 4000)
model = checkpoint2['model']

model.eval()
num_correct = 0
avg_val_loss = 0
for batch_num, (x, y) in enumerate(dev_dataloader):
    x, y = x.to(device), y.to(device)
    outputs = model(x)
    num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
    val_loss = criterion(outputs, y.long())
    avg_val_loss += val_loss.item()

accuracy = num_correct / len(dev_dataset)
avg_val_loss /= len(dev_dataset)
lr_scheduler.step(avg_val_loss)

print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch+1, num_correct / len(dev_dataset)))

### ! Computing Cosine similarity is implememnted in another notebook based on the hints given in the bootcamp!